# EDA

## Marta ---------------------------------------------------------

put all of your code between here and the next person's name only

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format = 'retina'

In [ ]:
noaa_data = pd.read_csv('data/noaa_data_clean.csv', index_col= False)

In [ ]:
noaa_data.columns

In [ ]:
noaa_data.describe()

In [ ]:
climate_cols = ['pcp', 'tavg', 'pdsi', 'phdi','zndx', 'pmdi', 'cdd', 'hdd', 'sp01', 'sp02', 'sp03', 'sp06', 'sp09',
               'sp12', 'sp24', 'tmin', 'tmax']

plt.figure(figsize=(17,23))
for i in range(len(climate_cols)):
    plt.subplot(6,3,i+1)
    plt.scatter(noaa_data['yearmonth'], noaa_data[climate_cols[i]], color = '#5f946e', marker = '.')
    plt.title(f"{climate_cols[i]}")
    
plt.savefig('./visuals/climate_scatters.png')

In [ ]:
plt.figure(figsize = (17,5))
sns.boxplot(data = noaa_data, x = 'state', y = 'pcp', palette= 'Greens')
plt.title('Precipitation Index by state');
plt.savefig('./visuals/pcp_by_state.png')

In [ ]:
plt.figure(figsize = (17,5))
sns.boxplot(data = noaa_data, x = 'state', y = 'tavg', palette= 'Greens')
plt.title('Temperature Index by state');
plt.savefig('./visuals/tavg_by_state.png')

In [ ]:
plt.figure(figsize = (17,5))
sns.boxplot(data = noaa_data, x = 'state', y = 'pdsi', palette= 'Greens')
plt.title('Drought Severity Index by state');
plt.savefig('./visuals/pdsi_by_state.png')

In [ ]:
plt.figure(figsize = (17,5))
sns.boxplot(data = noaa_data, x = 'state', y = 'phdi', palette= 'Greens')
plt.title('Hydrological Drought Index by state');
plt.savefig('./visuals/phdi_by_state.png')

In [ ]:
tmax_yr = noaa_data.groupby('year')['tmax'].mean().to_dict()
noaa_data['avg_yr_tmax'] = noaa_data['year'].apply(lambda x: tmax_yr[x])

tmin_yr = noaa_data.groupby('year')['tmin'].mean().to_dict()
noaa_data['avg_yr_tmin'] = noaa_data['year'].apply(lambda x: tmin_yr[x])

In [ ]:
plt.figure(figsize = (17,6))
sns.lineplot(data = noaa_data, x = 'year', y = 'tmax', color = '#5f946e', label = 'highest temp')
sns.lineplot(data = noaa_data, x = 'year', y = 'tmin', color = '#92e3a9', label = 'lowest temp')
plt.title('Lowest and highest temperature by year')
plt.legend();
plt.savefig('min_max_tmp_yr.png')

In [ ]:
noaa_data.tail()

## Jesse ---------------------------------------------------------

put all of your code between here and the next person's name only

In [ ]:
import sqlite3
import pandas as pd
from urllib.request import urlopen
import json
import pandas as pd
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
fire_map = pd.read_pickle('./data/fire_pickle.pkl')

In [ ]:
states_of_interest = ['CA','WA','OR','NV','AZ','NM','UT', 'CO', 'WY', 'ID', 'MT']

fire_map = fire_map[fire_map['STATE'].isin(states_of_interest)]

fire_map['STATE']=fire_map['STATE'].replace({'CA': '06', 
                       'WA': '53',
                      'OR': '41',
                      'NV': '32',
                      'AZ': '04',
                      'NM': '35',
                      'UT': '49',
                      'CO': '08',
                      'WY': '56',
                      'ID': '16',
                      'MT': '30',
                      })

fire_map['combined_fips'] =  fire_map['STATE'] + fire_map['FIPS_CODE']

In [ ]:
wildfire = fire_map.groupby(['combined_fips']).agg({'FIRE_SIZE': ['count']})

wildfire['FIPS_CODE'] = wildfire.index

wildfire.columns = ["FIPS_CODE", "FIRE_SIZE"]

wildfire.reset_index()

In [ ]:
wildfire_map = wildfire[['FIPS_CODE', 'count']]


fig = px.choropleth(wildfire_map, 
                    geojson=counties,
                    locations='FIPS_CODE',
                    color='count',
                           color_continuous_scale="Viridis",
                           range_color=(100, 6000),
                           scope="usa",
                           labels={'FIRE_SIZE':'FIRE_COUNT'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## CM ---------------------------------------------------------

put all of your code between here and the next person's name only

In [ ]:
  ### For Window's Users
import os
os.environ['PROJ_LIB'] = r'C:\Users\cmrjk\Anaconda3\pkgs\proj4-5.2.0-ha925a31_1\Library\share'
from mpl_toolkits.basemap import Basemap
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
fire_position = pd.read_csv('./data/fire_lat_lon.csv', index_col=False)
fire_position

In [ ]:
# Grabbing the lats and longs data
lats = []
lons = []
for i in fire_position.index:
    lats.append(fire_position['latitude'][i])
    lons.append(fire_position['longitude'][i])

# found this out on 
# https://makersportal.com/blog/2018/7/20/geographic-mapping-from-a-csv-file-using-python-and-basemap


# setting parameters for title and axes
font = {'family' : 'verdana',
        'size'   : 16}
matplotlib.rc('font', **font)

# How much to zoom from coordinates (in degrees)
zoom_scale = 3

# Setup the bounding box for the zoom and bounds of the map
bbox = [np.min(lats)-zoom_scale,np.max(lats)+zoom_scale,\
        np.min(lons)-zoom_scale,np.max(lons)+zoom_scale]

fig, ax = plt.subplots(figsize=(12,7))
plt.title("ASOS Continental United States (CONUS) Station Elevation")
# Define the projection, scale, the corners of the map, and the resolution.
m = Basemap(projection='merc',llcrnrlat=bbox[0],urcrnrlat=bbox[1],\
            llcrnrlon=bbox[2],urcrnrlon=bbox[3],lat_ts=10,resolution='i')

# Draw coastlines and fill continents and water with color
m.drawcoastlines()
m.fillcontinents(color='#CCCCCC',lake_color='lightblue')

# draw parallels, meridians, and color boundaries
m.drawparallels(np.arange(bbox[0],bbox[1],(bbox[1]-bbox[0])/5),labels=[1,0,0,0])
m.drawmeridians(np.arange(bbox[2],bbox[3],(bbox[3]-bbox[2])/5),labels=[0,0,0,1],rotation=15)
m.drawmapboundary(fill_color='lightblue')

# # format colors for elevation range
# alt_min = np.min(altitude)
# alt_max = np.max(altitude)
# cmap = plt.get_cmap('gist_earth')
# normalize = matplotlib.colors.Normalize(vmin=alt_min, vmax=alt_max)

# # plot elevations with different colors using the numpy interpolation mapping tool
# # the range [50,200] can be changed to create different colors and ranges
# for ii in range(0,len(altitude)):
#     x,y = m(lons[ii],lats[ii])
#     color_interp = np.interp(altitude[ii],[alt_min,alt_max],[50,200])
#     plt.plot(x,y,marker='o',markersize=6,color=cmap(int(color_interp)))

# # # format the colorbar 
# cax, _ = matplotlib.colorbar.make_axes(ax)
# cbar = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap,norm=normalize,label='Elevation')

x,y = m(lons,lats)
m.plot(x,y,'r*',markersize=2)
plt.title("Western Fire Distribution")


# save the figure and show it
plt.savefig('firemap_latlong.png', format='png', dpi=500,transparent=True)
plt.show()

## Kira ---------------------------------------------------------

put all of your code between here and the next person's name only